In [1]:
## Libraries need to calculate Moprho-VAE

import glob
import os
import warnings

import cv2  

import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns

from functions import *

warnings.simplefilter('ignore')

Using TensorFlow backend.
/home/tsutsumi/.conda/envs/TFGPU223_20201217/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/tsutsumi/.conda/envs/TFGPU223_20201217/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/tsutsumi/.conda/envs/TFGPU223_20201217/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.in

In [2]:
## GPU settings 
## This code is written in Tensorflow v1
### if you want to run this notebook, you should prepare GPU.

config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        visible_device_list="0", # specify GPU number
        allow_growth=True
    )
)
set_session(tf.Session(config=config))

# Load images 

In [3]:
# load figures 
X, y = load_data()
filelist = pd.read_csv("./new_Mandible_check.csv", encoding="SHIFT-JIS")


test_seed = 230
val_seed = 42
## Split data into train and test data.
X_train, y_train,Y_train, X_test, y_test , Y_test, group_train, group_test =  make_train_test(X,y,seed = test_seed)
## Split train data into val  and train data
X_train_2, y_train_2,Y_train_2, X_val, y_val ,Y_val, val_seed, group_train_2, group_val = \
make_val_train(X_train,y_train,group_train,seed = val_seed)
print(val_seed)

num_to_name ={0: 'Cercopithecidae',
 1: 'Cebidae',
 2: 'Lemuridae',
 3: 'Atelidae',
 4: 'Hylobatidae',
 5: 'Homonidae',
 6: 'Phocidae'}

y_name= np.array(pd.Series(y).replace(num_to_name))


print("train")
print(group_train)
print("test")
print(group_test) 
print("val")
print(group_val)

Train size :(195, 128, 128, 3), Test size:(99, 128, 128, 3)
Train2 size :(145, 128, 128, 3), val size:(50, 128, 128, 3)
42
train
['PRICT-998' 'PRICT-998' 'PRICT-990' 'PRICT-990' 'PRICT-1270' 'PRICT-1270'
 'PRICT-1266' 'PRICT-1266' 'PRICT-1264' 'PRICT-1264' 'PRICT-1256'
 'PRICT-1256' 'PRICT-1252' 'PRICT-1252' 'PRICT-1244' 'PRICT-1244'
 'PRICT-1242' 'PRICT-1242' 'PRICT-1238' 'PRICT-1238' 'PRICT-1141'
 'PRICT-1141' 'PRICT-1135' 'PRICT-1135' 'PRICT-1133' 'PRICT-1133'
 'PRICT-1098' 'PRICT-1098' 'PRICT-1062' 'PRICT-1062' 'PRICT-1054'
 'PRICT-1054' 'PRICT-1053' 'PRICT-1053' 'PRICT-1009' 'PRICT-1009'
 'PRICT-1008' 'PRICT-1008' 'PRICT-1006' 'PRICT-1006'
 'Mandrillus_leucophaeus_mandible_23169'
 'Mandrillus_leucophaeus_mandible_23169'
 'Mandrillus_leucophaeus_mandible_23168'
 'Mandrillus_leucophaeus_mandible_23168'
 'Mandrillus_leucophaeus_mandible_20085'
 'Mandrillus_leucophaeus_mandible_20085'
 'Mandrillus_leucophaeus_mandible_19986'
 'Mandrillus_leucophaeus_mandible_19986' 'M9754' 'M9754' 'M9

# Model setting 

In [4]:
### Model setting 
####  this csv file includes 10 tuned models.
#### Best model is 7-th model. 
arch_list = pd.read_csv('./architecture_list.csv')
num = 7
### ration of Reconstruction loss and Classification losss
### in this paper we set alpha = 0.1
## If you set alpha = 0 this matches VAE. 
alpha = 0.1 
### the dimension of latent space ζ
latent_dim = 3

### create Morpho-VAE model
model, z_mean, z_log_var, encoder, decoder, Classifier = create_model(arch_list.iloc[num,0], arch_list.iloc[num,1], list((arch_list.iloc[num,2:7]).astype(int)),3)

optimizer = arch_list.iloc[num,8]
model.compile(optimizer=optimizer, 
          loss = ['categorical_crossentropy',], 
          loss_weights = [alpha, 1 - alpha],
          metrics=['accuracy'])

NameError: name 'vae_loss' is not defined

## load Morpho-VAE's weights

In [4]:
## We load the weights of Morpho-VAE.
## If you want to train Morpho-VAE, ignore this cell and move to 2.2 train model section.
test_seed = 230
val_seed = 42

    
encoder_path = glob.glob("./WEIGHTS/Publish/*/{0}/encoder_*".format(test_seed))[0]
decoder_path = glob.glob("./WEIGHTS/Publish/*/{0}/decoder_*".format(test_seed))[0]
Classifier_path = glob.glob("./WEIGHTS/Publish/*/{0}/Classifier_*".format(test_seed))[0]
model_path = glob.glob("./WEIGHTS/Publish/*/{0}/model_*".format(test_seed))[0]

encoder.load_weights(encoder_path)
decoder.load_weights(decoder_path)
Classifier.load_weights(Classifier_path)
model.load_weights(model_path)

NameError: name 'encoder' is not defined

## load VAE's weights

In [ ]:
## VAE
test_seed = 227

arch_list = pd.read_csv('/home/tsutsumi/BONE/MORPHO-VAE/CODE/architecture_list.csv')
num = 4

VAE_model, z_mean, z_log_var, VAE_encoder, VAE_decoder, VAE_Classifier = create_model(arch_list.iloc[num,0], arch_list.iloc[num,1], list((arch_list.iloc[num,2:7]).astype(int)),3)


encoder_path = glob.glob("./WEIGHTS/Publish/20220211/{0}/encoder_*".format(test_seed))[0]
decoder_path = glob.glob(".WEIGHTS/Publish/20220211/{0}/decoder_*".format(test_seed))[0]
Classifier_path = glob.glob("./WEIGHTS/Publish/20220211/{0}/Classifier_*".format(test_seed))[0]
model_path = glob.glob("./WEIGHTS/Publish/20220211/{0}/model_*".format(test_seed))[0]

VAE_encoder.load_weights(encoder_path)
VAE_decoder.load_weights(decoder_path)
VAE_Classifier.load_weights(Classifier_path)
VAE_model.load_weights(model_path)

## train model

Note that if you want to train Morpho-VAE model, GPU enviroments is needed.
CPU environments will cost you a lot of time.

In [ ]:
model.fit(X_train_2,
                      [Y_train_2, X_train_2],
                      epochs=100,
                      batch_size=10,
                      validation_data=(X_val,[Y_val,X_val]),
                      callbacks = callbacks,
                     verbose = 2)

# Visualization 

## Cluster Separation

In [ ]:
## Morpho-VAE
X_predict_morphoVAE = encoder.predict(X)
plot_3d(X_predict_morphoVAE, y_name)

In [ ]:
## VAE
X_predict_VAE = VAE_encoder.predict(X)
plot_3d(X_predict_VAE, y_name)

In [ ]:
## PCA
d1, d2, d3, d4 = X.shape
X_flatten = X.reshape((d1, -1))
pca = PCA(n_components=3, svd_solver='arpack')
pca.fit(X_flatten)
feature = pca.transform(X_flatten)

plot_3d(feature, y_name)

## Reconstructiong and Generating Images from Latent Space

In [ ]:
## The value of PC3
zi = 0
## The size of rectangle
size = 10

x_predict = X_predict_morphoVAE.copy()
xyz = np.mean(x_predict, axis = 0)[:,np.newaxis]+ zi * pca.components_[2][:,np.newaxis] 

In [ ]:
## latent space and PCA plane
plot_latent_and_PCAplane(x_predict, y_name)

In [ ]:
## PCA plane
pca = PCA()
feature = pca.fit(x_predict)
feature = pca.transform(x_predict)

plot_2d(feature,y_name)

In [ ]:
## reconst grid images
reconst_grid_images(size,xyz,digit_size = 128,THR = 0.8)

## Visual Explanation of the Basis for Class Decisions

In [ ]:
fig = plt.figure(figsize=(5,10))
for family_num in [5]:
    for idx in range(np.where([y_test == family_num])[1].shape[0]):
        Num = np.where([y_test == family_num])[1][idx]
        family = sorted(set(y_name))[np.argmax(Y_train[Num])]
        for color,i,ch in zip(['G','B','R'],[0,1,2],[1,2,0]):
            ax = fig.add_subplot(np.where([y_test == family_num])[1].shape[0], 3, idx * 3+i +1)
            ax.axes.xaxis.set_visible(False)
            ax.axes.yaxis.set_visible(False)

            x, A,act_map = score_cam(num = Num , layer_name = '{}_conv_4'.format(color))
            A = cv2.resize(A,(128,128))
            A = np.maximum(A, 0)     
            A /= np.max(A)      
            ax.imshow(A,cmap='GnBu')
            ax.imshow(x[0,:,:,ch], alpha = 0.1)
plt.tight_layout()

## Reconstruction from Cropped Data

In [ ]:
name = 'M012'
size = 25

NAME = '{}.png'.format(name)
CROPPED = '../Fixed_figure/Crop_fig_{}/RGB/{}'.format(size,NAME)
NOT_CROPPED = '../New-RGB/{}'.format(NAME)
plt.figure(figsize=(15,5))
plt.subplot(131)
plt.imshow(cv2.imread(NOT_CROPPED))
plt.subplot(132)
plt.imshow(cv2.imread(CROPPED))
plt.subplot(133)
plt.imshow(decoder.predict(encoder.predict(cv2.imread(CROPPED)[np.newaxis]/255))[0])
print("True: {} , Predict: {}".format(y_test[group_test == name][0],np.argmax(Classifier.predict(cv2.imread(CROPPED)[np.newaxis]/255))))